In [1]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw,  PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [2]:
# data_path = "crab_data/0374"
data_path = "crab_data/1380"

In [3]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [4]:
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [5]:
skip_pointing = [False] * len(pointings)
skip_pointing[0] = True

In [6]:
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])

In [7]:
# Define Source and Spectrum
# ra, dec = 10, -40
# K, piv, index = 8e-3, 40, -2
ra, dec = 155., 75.
K, piv, index = 3e-3, 40, -1
    
pl = Powerlaw()
pl.piv = piv
pl.K = K
pl.index = index
component1 = SpectralComponent("pl", shape=pl)
source = PointSource("Test", ra=ra, dec=dec, components=[component1])

emod = np.geomspace(10, 3000, 50)
spec = source(emod)
spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2

In [8]:
# Generate Source Counts

assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)

source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], ra, dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mat = sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1))
    
    count_rates = np.dot(spec_binned, resp_mat)
    
    for d_i, d in enumerate(dets):
        index = p_i * 85 + d
        source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[index])    


Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)


In [32]:
# Save Data for PySpi

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts.copy()
updated_counts["COUNTS"] += source_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path}/evts_det_spec.fits")


In [9]:
#  Spimodfit backgroud for PySpi
# Manually copy energy_boundaries.fits

data_path2 = "crab_data/1380_spimodfit_bkg"

In [10]:


with fits.open(f"{data_path2}/bg-e0020-0600/output_bgmodel-conti.fits.gz") as file:
    t = Table.read(file[1])
    conti = t["COUNTS"]
    
with fits.open(f"{data_path2}/bg-e0020-0600/output_bgmodel-lines.fits.gz") as file:
    t = Table.read(file[1])
    lines = t["COUNTS"]


In [11]:
################

spimodfit_folder = "crab_data/1380_spimodfit"

with fits.open(f"{spimodfit_folder}/pointing.fits.gz") as file:
    t = Table.read(file[1])
    
spimodfit_pointings = np.array(t["PTID_ISOC"])

with fits.open(f"{spimodfit_folder}/energy_boundaries.fits.gz") as file:
    t = Table.read(file[1])
    
new_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [12]:
# with fits.open(f"{data_path2}/energy_boundaries.fits") as file:
#     t = Table.read(file[1])
#     new_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Energy Indices
energy_indices = []
for e in new_energy_bins:
    temp = np.argwhere(energy_bins==e)
    if len(temp)>0:
        energy_indices.append(temp[0][0])

# spimodfit_pointings = ["03740001", "03740002", "03740003", "03740004", "03740005", "03740006", "03740007", "03740008", "03740009", "03740010", "03740011", "03740012", "03740013", "03740014", "03740015", "03740016", "03740017", "03740018", "03740019", "03740020", "03740021", "03740022", "03740023", "03740024", "03740025", "03740026", "03740027", "03740028", "03740029", "03740030", "03740031", "03740032", "03740033", "03740035", "03740036", "03740037", "03740038", "03740039", "03740040", "03740041", "03740042", "03740043", "03740045"]

# Pointing Indices
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    pointing_indices = []
    pointing_indices_full = []
    for i, p in enumerate(spimodfit_pointings):
        temp = np.argwhere(t["PTID_ISOC"]==p)
        if len(temp)>0:
            pointing_indices.append(temp[0][0])
            for j in range(85):
                pointing_indices_full.append(85*temp[0][0] + j)
            

        



In [16]:
source_counts[85*25,:15]

array([ 3,  3,  1,  3, 19, 17, 10, 13, 10, 11, 15, 10, 22, 20, 13],
      dtype=uint32)

In [18]:
new_source_counts = np.zeros((len(source_counts), len(new_energy_bins)-1))

for i in range(len(energy_indices)-1):
    new_source_counts[:,i] = np.sum(source_counts[ : , energy_indices[i] : energy_indices[i+1]], axis=1)
    
new_source_counts = new_source_counts[pointing_indices_full]

total_counts = new_source_counts.copy()

for i in range(len(pointing_indices)):
    total_counts[85*i : 85*i + 19, :] += (np.random.poisson(np.abs(lines[i*19: (i+1)*19])) * np.sign(lines[i*19: (i+1)*19])
                                          + np.random.poisson(np.abs(conti[i*19: (i+1)*19])) * np.sign(conti[i*19: (i+1)*19]))

In [19]:
with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts[pointing_indices_full]
updated_counts["COUNTS"] = total_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path2}/evts_det_spec.fits")

In [20]:
# dead_time.fits
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    
dt = t[pointing_indices_full]

hdu = fits.BinTableHDU(data=dt, name="SPI.-OBS.-DTI")
hdu.writeto(f"{data_path2}/dead_time.fits")

In [21]:
# pointing.fits
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
ps = t[pointing_indices]

hdu = fits.BinTableHDU(data=ps, name="SPI.-OBS.-PNT")
hdu.writeto(f"{data_path2}/pointing.fits")

In [22]:
# spimodfit_folder = "crab_data/0374_spimodfit"
spimodfit_folder = "crab_data/1380_spimodfit"

if not os.path.exists(spimodfit_folder):
    os.mkdir(spimodfit_folder)

spimodfit_pointings = ["03740001", "03740002", "03740003", "03740004", "03740005", "03740006", "03740007", "03740008", "03740009", "03740010", "03740011", "03740012", "03740013", "03740014", "03740015", "03740016", "03740017", "03740018", "03740019", "03740020", "03740021", "03740022", "03740023", "03740024", "03740025", "03740026", "03740027", "03740028", "03740029", "03740030", "03740031", "03740032", "03740033", "03740035", "03740036", "03740037", "03740038", "03740039", "03740040", "03740041", "03740042", "03740043", "03740045"]


spimodfit_energy_bins = [20.0, 21.5, 23.5, 25.5, 27.5, 30.0, 32.5, 35.5, 38.5, 42.0, 45.5, 49.5, 54.0, 58.5, 63.5, 69.0, 75.0, 81.5, 89.0, 96.5, 105.0, 114.0, 124.0, 134.5, 146.0, 159.0, 172.5, 187.5, 204.0, 221.5, 240.5, 261.5, 284.0, 308.5, 335.5, 364.5, 396.0, 430.0, 467.5, 508.0, 514.0, 600.0,]

In [23]:
with fits.open(f"{spimodfit_folder}/pointing.fits.gz") as file:
    t = Table.read(file[1])
    
spimodfit_pointings = np.array(t["PTID_ISOC"])

with fits.open(f"{spimodfit_folder}/energy_boundaries.fits.gz") as file:
    t = Table.read(file[1])
    
spimodfit_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])


In [24]:
# Pointing Indices
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    p_indices = []
    p_rel_indices = []
    for i, p in enumerate(spimodfit_pointings):
        temp = np.argwhere(t["PTID_ISOC"]==p)
        if len(temp)>0:
            p_indices.append(temp[0][0])
            p_rel_indices.append(i)

# Energy Indices
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    e_indices = []
    for e in spimodfit_energy_bins:
        temp = np.argwhere(t["E_MIN"]==e)
        if len(temp)>0:
            e_indices.append(temp[0][0])
            
# Single Event Indices
se_indices = []
for i in p_indices:
    for j in range(19):
        se_indices.append(i*85 + j)
        
# Relative Pointing Deterctor Indices
rd_indices = []
for i in p_rel_indices:
    for j in range(19):
        rd_indices.append(i*19 + j)

In [25]:
# evts_det_spec.fits.gz
with fits.open(f"{data_path}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
with fits.open(f"{spimodfit_folder}/evts_det_spec_old.fits.gz") as file:
    d = Table.read(file[1])
    h = file[1].header
    
eds_temp = t[se_indices]


for i in range(len(e_indices)-1):
    d["COUNTS"][:,i] = np.sum(eds_temp["COUNTS"][ : , e_indices[i] : e_indices[i+1]], axis=1)
    
hdu = fits.BinTableHDU(data=d, name="SPI.-OBS.-DSP")
hdu.header = h
hdu.writeto(f"{spimodfit_folder}/evts_det_spec.fits.gz")



OSError: File crab_data/1380_spimodfit/evts_det_spec.fits.gz already exists. If you mean to replace it then use the argument "overwrite=True".

In [27]:
# spimodfit with spimodfit bkg
with fits.open(f"{data_path2}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
with fits.open(f"{spimodfit_folder}/evts_det_spec_old.fits.gz") as file:
    d = Table.read(file[1])
    h = file[1].header

smf_bkg_indices = []
for i in range(len(pointing_indices)):
    for j in range(19):
        smf_bkg_indices.append(i*85 + j)

d["COUNTS"] = np.array(t["COUNTS"][smf_bkg_indices], dtype=np.uint32)

hdu = fits.BinTableHDU(data=d, name="SPI.-OBS.-DSP")
hdu.header = h
hdu.writeto(f"{spimodfit_folder}/w_smf_bkg/evts_det_spec.fits.gz")

In [28]:
hdu.data

FITS_rec([([        279,         708,         887,         416,         389,         498,         486,         488,         532,         602,         691,        1696,        2884,        2622,        4555,         844,         860,         868,        2091,        1324,         924,        1087,        1222,        3932,        1429,        1412,        2107,        5805,        1436,        1407,        1419,        1483,        1606,        1453,        1353,        1247,        1368,        1653,        2035,         310,        2473], [ 18.193405,  34.29286 ,  40.718548,  25.612497,  24.718414,  23.194826,  25.768198,  25.23886 ,  26.532999,  26.870058,  29.631065,  57.95688 ,  67.992645,  67.48333 ,  92.99462 ,  34.176014,  32.109188,  33.42155 ,  58.189346,  44.844173,  35.142567,  37.86819 ,  40.93898 ,  85.32877 ,  43.462627,  45.628937,  58.30952 , 104.302444,  45.40925 ,  46.249325,  45.67275 ,  47.937458,  50.1996  ,  50.348785,  47.549973,  47.042534,  48.466484,  50.56678

In [31]:
with fits.open(f"{data_path}/spi/w_smf_bkg/evts_det_spec.fits.gz") as file:
    t = Table.read(file[1])
    
t

COUNTS,STAT_ERR
uint32[41],float32[41]
127 .. 412,7.8740077 .. 29.051678
74 .. 382,8.062258 .. 28.01785
0 .. 0,0.0 .. 0.0
101 .. 423,8.6602545 .. 28.231188
122 .. 397,7.2111025 .. 27.748875
95 .. 412,7.2111025 .. 28.035692
103 .. 425,7.615773 .. 29.103264
52 .. 376,7.9372535 .. 28.600698
78 .. 420,7.8740077 .. 27.802877


In [ ]:
# rest not necessary

In [14]:
# energy_boundaries.fits.gz
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])

eb = t[e_indices[:-1]]
eb["E_MAX"] = t["E_MIN"][e_indices[1:]]

hdu = fits.BinTableHDU(data=eb, name="SPI.-EBDS-SET")
hdu.header["EBIN_NUM"] = len(e_indices[:-1])
hdu.writeto(f"{spimodfit_folder}/energy_boundaries.fits.gz", overwrite=True)

In [19]:
# dead_time.fits.gz
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    
dt = t[se_indices]

hdu = fits.BinTableHDU(data=dt, name="SPI.-OBS.-DTI")
hdu.header["ISOC_NUM"] = len(p_indices)
hdu.header["PT_NUM"] = len(p_indices)
hdu.header["DET_NUM"] = 19
hdu.writeto(f"{spimodfit_folder}/dead_time.fits.gz", overwrite=True)

In [21]:
# pointing.fits.gz
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
ps = t[p_indices]

hdu = fits.BinTableHDU(data=ps, name="SPI.-OBS.-PNT")
hdu.header["ISOC_NUM"] = len(p_indices)
hdu.header["PT_NUM"] = len(p_indices)
hdu.writeto(f"{spimodfit_folder}/pointing.fits.gz", overwrite=True)

In [23]:
# gti.fits.gz
with fits.open(f"{data_path}/gti.fits") as file:
    t = Table.read(file[1])

gt = t[se_indices]

hdu = fits.BinTableHDU(data=gt, name="SPI.-OBS.-GTI")
hdu.header["ISOC_NUM"] = len(p_indices)
hdu.header["PT_NUM"] = len(p_indices)
hdu.header["DET_NUM"] = 19
hdu.writeto(f"{spimodfit_folder}/gti.fits.gz", overwrite=True)

In [6]:
# simulated source
with fits.open(f"{data_path}/cat_crab.fits.gz") as file:
    t = Table.read(file[1])
    
ns = t.copy()
ns["RA_OBJ"] = 10.
ns["DEC_OBJ"] = -40.
ns["NAME"] = "SimSource"
ns["CLASS"] = 0
ns["ISGRI_FLAG"] = 1
ns["SPI_FLUX_1"] = 0.
ns["SPI_FLUX_2"] = 0.
ns["ISGR_FLUX_1"] = 0.
ns["ISGR_FLUX_2"] = 0.
ns["PICS_FLUX_1"] = 0.
ns["PICS_FLUX_2"] = 0.
ns["JEMX_FLUX_1"] = 0.
ns["JEMX_FLUX_2"] = 0.
ns["BIRD_SOURCE_ID"] = "J000000.0+000000"
ns["SOURCE_ID"] = "SimSource"

hdu = fits.BinTableHDU(data=ns, name="SPI.-SRCL-CAT")
hdu.writeto(f"{spimodfit_folder}/cat_sim.fits.gz", overwrite=True)


In [31]:
# scw.fits.gz

with fits.open(f"{data_path}/scw.fits.gz") as file:
    t = Table.read(file[1])
    
sw = t[p_rel_indices]

hdu = fits.BinTableHDU(data=ns, name="SPI.-SCW.-CAT")
hdu.writeto(f"{spimodfit_folder}/scw.fits.gz", overwrite=True)

XTENSION= 'BINTABLE'           / Binary table extension                         BITPIX  =                    8 / 8-bit bytes                                    NAXIS   =                    2 / 2-dimensional binary table                     NAXIS1  =                 7189 / width of table in bytes                        NAXIS2  =                   43 / number of rows in table                        PCOUNT  =                    0 / size of special data area                      GCOUNT  =                    1 / one data group (required keyword)              TFIELDS =                  349 / number of fields in each row                   TTYPE1  = 'ScwID   '           / Label for this field                           TFORM1  = '12A     '           / format of field                                TTYPE2  = 'Revolution'         / Label for this field                           TFORM2  = '1I      '           / format of field                                TZERO2  =                32768 / offset 